# iFLYTEK：[科大讯飞X光安检图像识别2021挑战赛](http://challenge.xfyun.cn/topic/info?type=Xray-2021)（非官方baseline）

**<font color=red>注意：本项目不提供赛题数据集，若训练，可通过上方链接从官网报名下载数据集</font>**

## 一、赛事背景

X光安检机是目前我国使用最广泛的安检技术手段，广泛应用于城市轨交、铁路、机场、重点场馆、物流寄递等场景。使用人工智能技术，辅助一线安检员进行X光安检判图，可以有效降低因为人员疲劳或注意力不集中带来的漏报等问题。但在实际场景中，因物品的多样性、成像角度、遮挡等问题，为算法的开发带来了一定的挑战。

讯飞看见X光智能安检信息化解决方案是针对安检场景推出的智能化、信息化解决方案，旨在将传统的安检业务场景，通过人工智能、大数据等先进技术手段进行赋能，辅助一线安检检查工作，为安检管理方提供量化数据与大数据预警等更多技术管理工具。

## 二、赛事任务

本次大赛以讯飞看见智慧安检提供的X光安检图像数据作为训练样本，参赛选手需基于提供的样本构建模型，对X光安检图像中的待识别物品进行检测。

## 三、数据说明

此次比赛分为初赛和复赛两个阶段，初赛提供1）带标注的训练数据，即待识别物品在包裹中的X光图像及其标注文件；2）不带标注的测试数据；复赛在初赛的基础上再提供1）无标注训练数据即包裹X光图像（其中有的包裹包含待识别物品）；2）部分待识别物品X光图像（无背景）。

本次比赛标注文件中的类别为12类，包括

刀（label为knife）、剪刀（label为scissors）、尖锐工具（label为sharpTools）、甩棍（label为expandableBaton）、小玻璃瓶（label为smallGlassBottle）、电棍（label为electricBaton）、塑料饮料瓶（label为plasticBeverageBottle）、带喷嘴塑料瓶（label为plasticBottleWithaNozzle）电子设备（label为electronicEquipment）、电池（label为battery）、公章（label为seal）、伞（label为umbrella）。

12类待识别物品的X光成像示意图如下图所示。 

![](https://ai-studio-static-online.cdn.bcebos.com/8267237a2542468083049797c09dff3311ffdf94f7234bddaf35a8af09e59eec)

初赛提供的X光图像及其矩形框标注的文件按照数据来源存放在不同的文件夹中，图像文件采用jpg格式，标注文件采用xml格式，各字段含义参照voc数据集。VOC各字段含义对应表为：

├── filename 文件名

├── size 图像尺寸

├── width 图像宽度

├── height 图像高度

└──depth 图像深度，一般为3表示是彩色图像

└── object图像中的目标，可能有多个

├── name 该目标的标签名称

└── bndbox该目标的标注框

├── xmin 该目标的左上角宽度方向坐标

├── ymin 该目标的左上角高度方向坐标

├── xmax 该目标的右下角宽度方向坐标

└── ymax 该目标的右下角高度方向坐标

## 四、比赛思路

该赛题很明显是实现目标检测，所以使用`PaddleDetection`是最方便的，数据集是`VOC`格式，也不需要进行格式转换。将数据集划分训练集和验证集，然后用`PaddleDetection`直接训练就可以了。

PaddleDetection文档可见：[欢迎使用 PaddleDetection!](https://paddledetection.readthedocs.io/index.html)

PaddleDetection官方源码：[PaddleDetection](https://gitee.com/paddlepaddle/PaddleDetection)

In [ ]:
#首先以8：2划分一下训练集和验证集
import os
import random

Root='iFLYTEK_Xray/train'

with open('train.txt','w',encoding='utf-8') as fw:
    for path in os.listdir(Root):
        path=os.path.join(Root,path)
        files=os.listdir(path)
        for file in files:
            if file.endswith('.jpg'):
                fw.write(os.path.join(path,file)+' '+ os.path.join(path,'XML/'+os.path.basename(file).split('.')[0]+'.xml\n'))

with open('train.txt','r',encoding='utf-8') as fr:
    lines= fr.readlines()
    random.shuffle(lines)
with open('train.txt','w',encoding='utf-8') as fw:
    fw.writelines(lines[:int(len(lines)*0.8)])
with open('eval.txt','w',encoding='utf-8') as fw:
    fw.writelines(lines[int(len(lines)*0.8):])

## 五、使用HRNet进行训练

<font size=3>Paper：[Deep High-Resolution Representation Learning for Human Pose Estimation](https://arxiv.org/abs/1902.09212)</font>

![](https://ai-studio-static-online.cdn.bcebos.com/493e4cd56e824f9abca0123ee31426ba159639eff192466e91173e7dc992a964)

<font size=3>HRNet从一个高分辨率子网作为第一级开始，逐步将高分辨率子网和低分辨率子网逐一相加，形成多个级，并将多分辨率子网连接起来同时进行，使网络在整个过程中保持高分辨率的表示。因此，预测的关键点热图可能更精确，空间上更精确。</font>

![](https://ai-studio-static-online.cdn.bcebos.com/b7a85c5544da4d5794f4d735de06037498f8090b01984bb0af2a4f105cda4821)


#### **（1）对配置文件进行修改**

主要修改：`configs/datasets/voc.yml`、`configs/hrnet/faster_rcnn_hrnetv2p_w18_1x_coco.yml`、`configs/hrnet/_base_/faster_rcnn_hrnetv2p_w18.yml`、`configs/faster_rcnn/_base_/optimizer_1x.yml`以及`configs/faster_rcnn/_base_/faster_fpn_reader.yml`

这里仅仅将图片Resize到[640, 640]

#### **（2）模型训练**

相关训练配置可见：[入门使用](https://paddledetection.readthedocs.io/tutorials/GETTING_STARTED_cn.html)

In [ ]:
!python PaddleDetection/tools/train.py \
        #-r output/faster_rcnn_hrnetv2p_w18_1x_coco/best_model

In [ ]:
#删除多余模型文件
import os
Root='output/faster_rcnn_hrnetv2p_w18_1x_coco'
path = os.listdir(Root)
for file in path:
    if not file.startswith('best_model'):
        os.remove(os.path.join(Root,file))

## 六、模型预测

模型预测代码主要针对比赛[作品提交要求](http://challenge.xfyun.cn/topic/info?type=Xray-2021)进行了修改，修改代码主要为：`tools\infer.py`、`ppdet\utils\visualizer.py`。

#### **（1）生成图片预测文本**

这里要注意，由于赛题提交的json文件没有图片名，只是一串数组，所以预测图片必须要按照从小到大的顺序进行排列。

![](https://ai-studio-static-online.cdn.bcebos.com/6f11a6997f13498bbe79325b0abe441aa86309fc991e4d0ea9a3859b40002ad3)


In [ ]:
import os

Root='iFLYTEK_Xray/test'
file_list = sorted(os.listdir(Root),key=lambda x:os.path.basename(x).split('.')[0])

with open('test.txt','w',encoding='utf-8') as fp:
    for path in file_list:
        fp.write(os.path.join(Root,path)+'\n')

#### **（2）对图片进行预测**

预测配置可见：[训练/评估/预测](https://paddledetection.readthedocs.io/tutorials/GETTING_STARTED_cn.html#id2)

In [ ]:
!python PaddleDetection/tools/infer.py \
                -c PaddleDetection/configs/hrnet/faster_rcnn_hrnetv2p_w18_2x_coco.yml \
                --output_dir=infer_output/ \
                --draw_threshold=0.5 \
                -o weights=output/faster_rcnn_hrnetv2p_w18_1x_coco/best_model

## 七、预测结果展示

![](https://ai-studio-static-online.cdn.bcebos.com/fee03f3bdd1b49ff9c6a938767ceac976391cce229b146acb35f612a3363f429)

提交文件是生成的result.json，如下:

![](https://ai-studio-static-online.cdn.bcebos.com/6baedee5fbf345c2a79a05f4fdea6963011fec00256e400cb22803b337aaf44f)


## 八、总结

使用HRNet只训练了20epoch就达到了比较理想的结果，但是仍有很大的提升空间，大家可以通过调整参数提升模型的精度。

>本人武汉科技大学信息安全专业毕业，今年考入本校研究生，在CV和NLP上都有兴趣~

>我在AI Studio上获得钻石等级，点亮8个徽章

>欢迎关注[我的主页](https://aistudio.baidu.com/aistudio/personalcenter/thirdview/71231)

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 